In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import itertools

In [2]:
numurbl = 10
lsmlat = 1 
lsmlon = 1
numrad = 2
nlevurb = 5 # use 5!!!
GRIDNAME=['AU-Pre', 'US-Mi1', 'US-Wes']
num_city = len(GRIDNAME)
data_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/CESM/bakeup/UrbanPlumber/'
path = data_dir + 'input_files/'
output_path = data_dir + 'sensitivity_input/'
lcz_type = [0,0,0,0,0,100,0,0,0,0]

default = {}
for i in range(num_city):
    folder = GRIDNAME[i] + '/'
    filename = 'surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c230710.nc'
    ds = xr.open_dataset(path + folder+ filename)
    default['ds_' + GRIDNAME[i]] = xr.Dataset()
    for var_name in ds.variables:
        variable = ds[var_name]
        if 'numurbl' in variable.dims:
            default['ds_' + GRIDNAME[i]][var_name] = variable

datasets = {} # output container
for i in range(num_city):
    folder = GRIDNAME[i] + '/'
    filename = 'surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c230710.nc'
    ds = xr.open_dataset(path + folder+ filename)
    datasets['ds_' + GRIDNAME[i]] = xr.Dataset()
    for var_name in ds.variables:
        variable = ds[var_name]
        if 'numurbl' not in variable.dims:
            datasets['ds_' + GRIDNAME[i]][var_name] = variable          

In [18]:
var0 = ['WALL_TO_PLAN_AREA_RATIO', 'WIND_HGT_CANYON', 'T_BUILDING_MIN', 'NLEV_IMPROAD', 'PCT_URBAN']
var1 = ['ALB_IMPROAD_DIF','ALB_PERROAD_DIF','ALB_ROOF_DIF',  'ALB_WALL_DIF', 
        'EM_IMPROAD', 'EM_PERROAD', 'EM_ROOF', 'EM_WALL']
var1_1 = ['ALB_IMPROAD_DIR', 'ALB_PERROAD_DIR', 'ALB_ROOF_DIR', 'ALB_WALL_DIR']
var2 = ['CANYON_HWR', 'HT_ROOF', 'THICK_ROOF', 
        'THICK_WALL', 'WTLUNIT_ROOF', 'WTROAD_PERV']   
var3 = ['TK_IMPROAD', 'TK_ROOF', 'TK_WALL', 
        'CV_ROOF', 'CV_WALL', 'CV_IMPROAD']

ai_cesm = [0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14]
ap_cesm = [0.08, 0.08 , 0.08 , 0.08 , 0.08 , 0.08 , 0.08 , 0.08 , 0.08 , 0.08]
ar_cesm = [0.23, 0.28, 0.25, 0.23, 0.23, 0.23, 0.25 , 0.28 , 0.23 , 0.20]
aw_cesm = [0.30, 0.25 , 0.25 , 0.30 , 0.30 , 0.30 , 0.35 , 0.30 , 0.30 , 0.25]
ca_cesm = [2.50, 1.25 , 1.25 , 0.75 , 0.50 , 0.50 , 0.90 , 0.20 , 0.15 , 0.35]
ht_cesm = [37.50 , 17.50 , 6.50 , 37.50 , 17.50 , 6.50 , 3.00 , 6.50 , 6.50 , 10.00]
wi_cesm = np.array(ht_cesm)/2
wtr_cesm = [0.53 , 0.61 , 0.65 , 0.46 , 0.43 , 0.50 , 0.88 , 0.47 , 0.50 , 0.45]
wtp_cesm = [0.10 , 0.20 , 0.33 , 0.50 , 0.43 , 0.43 , 0.60 , 0.25 , 0.82 , 0.60]
li_cesm = [0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8] # TK imperoad
lr_cesm = [1.70 , 1.70 , 1.09 , 1.25 , 1.70 , 1.09 , 1.09 , 1.07 , 1.09 , 2.00] # TK roof: thermal conductivity
lw_cesm = [1.27 , 2.60 , 1.66 , 1.45 , 1.88 , 1.66 , 1.00 , 1.07 , 1.66 , 1.42 ] # TK wall
cr_cesm = [1.32 , 1.32 , 1.32 , 1.80 , 1.32 , 1.32 , 2.00 , 2.11 , 1.32 , 2.00] # CV_ROOF
cw_cesm = [1.54 , 1.54 , 1.54 , 2.00 , 1.54 , 1.54 , 2.00 , 2.11 , 1.54 , 1.59] # CV_WALL
ci_cesm = [1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80] # CV_IMPROAD: volumetric heat capacity
nlev = [3,2,2,2,3,2,2,2,2,2] # NLEV_IMPROAD
zr_cesm = [0.30 , 0.30 , 0.20 , 0.30 , 0.25 , 0.15 , 0.10 , 0.12 , 0.15 , 0.10] # THICK_ROOF
zw_cesm = [0.30 , 0.25 , 0.25 , 0.20 , 0.20 , 0.20 , 0.10 , 0.20 , 0.20 , 0.10] # 'THICK_WALL
ei_cesm = [0.91, 0.91, 0.91, 0.91, 0.91, 0.91, 0.88, 0.91, 0.91, 0.91]
ep_cesm = [0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
er_cesm = [0.91, 0.91, 0.91, 0.91, 0.91, 0.91, 0.88, 0.91, 0.91, 0.91] # emissivity
ew_cesm = [0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90]
tmin = [291, 287, 287, 291, 287, 287, 287, 287, 287, 287] # T_BUILDING_MIN
tmin_c = np.array(tmin)- 273.15

crr_cesm = [x * 1000000 for x in cr_cesm]
cww_cesm = [x * 1000000 for x in cw_cesm]
cii_cesm = [x * 1000000 for x in ci_cesm]

ucp1_cesm = [ai_cesm, ap_cesm, ar_cesm, aw_cesm, 
             ei_cesm, ep_cesm, er_cesm, ew_cesm] 
ucp2_cesm = [ca_cesm, ht_cesm, zr_cesm, zw_cesm, wtr_cesm, wtp_cesm]
ucp3_cesm = [li_cesm, lr_cesm, lw_cesm, crr_cesm, cww_cesm, cii_cesm]

for i in range(num_city):
    for j in var0:
        datasets['ds_' + GRIDNAME[i]][j] = (('numurbl', 'lsmlat', 'lsmlon'), 
                                            np.zeros((numurbl, lsmlat, lsmlon)))  
    for j in var2:
        datasets['ds_' + GRIDNAME[i]][j] = (('numurbl', 'lsmlat', 'lsmlon'), 
                                            np.zeros((numurbl, lsmlat, lsmlon))) 
    for j , varname in enumerate(var1):
        if j < 4:
            datasets['ds_' + GRIDNAME[i]][varname] = (('numrad', 'numurbl', 'lsmlat', 'lsmlon'), 
                                                      np.zeros((numrad, numurbl, lsmlat, lsmlon))) 
        else:
            datasets['ds_' + GRIDNAME[i]][varname] = (('numurbl', 'lsmlat', 'lsmlon'), 
                                                      np.zeros((numurbl, lsmlat, lsmlon)))
    for j in var1_1:    
        datasets['ds_' + GRIDNAME[i]][j] = (('numrad','numurbl', 'lsmlat', 'lsmlon'), 
                                            np.zeros((numrad, numurbl, lsmlat, lsmlon)))   
    for j in var3:
        datasets['ds_' + GRIDNAME[i]][j] = (('nlevurb','numurbl', 'lsmlat', 'lsmlon'), 
                                            np.zeros((nlevurb, numurbl, lsmlat, lsmlon)))    

In [15]:
# morphological parameters
# CESM LCZ-table export surface data
morphological_combinations = list(itertools.product([1.2, 0.8], repeat=len(var2)))

for i in range(len(GRIDNAME)):
    for m in range(10):
        # use the default 
        datasets['ds_' + GRIDNAME[i]]['WALL_TO_PLAN_AREA_RATIO'][m, :, :] = default['ds_' + GRIDNAME[i]]['WALL_TO_PLAN_AREA_RATIO'][2, :, :]
        datasets['ds_' + GRIDNAME[i]]['NLEV_IMPROAD'][m, :, :] = nlev[m]
        datasets['ds_' + GRIDNAME[i]]['PCT_URBAN'][m, :, :] = lcz_type[m]
        datasets['ds_' + GRIDNAME[i]]['T_BUILDING_MIN'][m, :, :] = tmin[m]
        
    for j in range(len(var1)):
        for m in range(10):
            if j < 4:
                datasets['ds_' + GRIDNAME[i]][var1[j]][:, m, :, :] = ucp1_cesm[j][m]
                datasets['ds_' + GRIDNAME[i]][var1_1[j]][:, m, :, :] = ucp1_cesm[j][m]
            else:
                datasets['ds_' + GRIDNAME[i]][var1[j]][m, :, :] = ucp1_cesm[j][m]
    
    for j in range(len(var3)):
        for m in range(10):
            datasets['ds_' + GRIDNAME[i]][var3[j]][:, m, :, :] = ucp3_cesm[j][m]            
            
    for label, change_factors in enumerate(morphological_combinations):
        modified_datasets = {}
        #label = '_'.join(['{}_{}'.format(var2[j],factor) for j, factor in enumerate(change_factors)])
        #label = ''.join(['D' if factor == 1.2 else 'X' for factor in change_factors])
        
        for j, factor in enumerate(change_factors):
            for m in range(10):
                modified_data = ucp2_cesm[j][m] * factor
                datasets['ds_' + GRIDNAME[i]][var2[j]][m, :, :] = modified_data
        for m in range(10):
            datasets['ds_' + GRIDNAME[i]]['WIND_HGT_CANYON'][m, :, :] = np.array(datasets['ds_' + GRIDNAME[i]][var2[1]][m, :, :])/2
        output = output_path + GRIDNAME[i] + '/surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c240510_morp_' + str(label) + '.nc'
        output_dir = os.path.dirname(output)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        if os.path.exists(output):
            os.remove(output)
        datasets['ds_' + GRIDNAME[i]].to_netcdf(output)                   

KeyError: 'WALL_TO_PLAN_AREA_RATIO'

In [6]:
# radiative parameters
radiative_combinations = list(itertools.product([1.2, 0.8], repeat=8))

for i in range(len(GRIDNAME)):
    for m in range(10):
        datasets['ds_' + GRIDNAME[i]]['WALL_TO_PLAN_AREA_RATIO'][m, :, :] = default['ds_' + GRIDNAME[i]]['WALL_TO_PLAN_AREA_RATIO'][2, :, :]
        datasets['ds_' + GRIDNAME[i]]['NLEV_IMPROAD'][m, :, :] = nlev[m]
        datasets['ds_' + GRIDNAME[i]]['PCT_URBAN'][m, :, :] = lcz_type[m]
        datasets['ds_' + GRIDNAME[i]]['T_BUILDING_MIN'][m, :, :] = tmin[m]
        datasets['ds_' + GRIDNAME[i]]['WIND_HGT_CANYON'][m, :, :] = wi_cesm[m]
        
    for j in range(len(var2)):
        for m in range(10):
            datasets['ds_' + GRIDNAME[i]][var2[j]][m, :, :] = ucp2_cesm[j][m]
              
    for j in range(len(var3)):
        for m in range(10):
            datasets['ds_' + GRIDNAME[i]][var3[j]][:, m, :, :] = ucp3_cesm[j][m]
                   
    for label, change_factors in enumerate(radiative_combinations): # for albedo
        modified_datasets = {}
        for j, factor in enumerate(change_factors):
            for m in range(10):
                if j < 4:
                    modified_data = ucp1_cesm[j][m] * factor
                    datasets['ds_' + GRIDNAME[i]][var1[j]][:, m, :, :] = modified_data
                    datasets['ds_' + GRIDNAME[i]][var1_1[j]][:, m, :, :] = modified_data
                else:
                    if factor==0.8: 
                        factor_for_emissivity=factor + 0.18
                    elif factor==1.2: 
                        factor_for_emissivity=factor - 0.18
                    modified_data = ucp1_cesm[j][m] * factor_for_emissivity    
                    datasets['ds_' + GRIDNAME[i]][var1[j]][m, :, :] = modified_data
                
        output = output_path + GRIDNAME[i] + '/surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c240510_radi_' + str(label) + '.nc'
        output_dir = os.path.dirname(output)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        if os.path.exists(output):
            os.remove(output)
        datasets['ds_' + GRIDNAME[i]].to_netcdf(output)                          

In [7]:
# thermal parameters
# CESM LCZ-table export surface data
thermal_combinations = list(itertools.product([1.2, 0.8], repeat=len(var3)))
for i in range(len(GRIDNAME)):
    for m in range(10):
        # use the default 
        datasets['ds_' + GRIDNAME[i]]['WALL_TO_PLAN_AREA_RATIO'][m, :, :] = default['ds_' + GRIDNAME[i]]['WALL_TO_PLAN_AREA_RATIO'][2, :, :]
        datasets['ds_' + GRIDNAME[i]]['NLEV_IMPROAD'][m, :, :] = nlev[m]
        datasets['ds_' + GRIDNAME[i]]['PCT_URBAN'][m, :, :] = lcz_type[m]
        datasets['ds_' + GRIDNAME[i]]['T_BUILDING_MIN'][m, :, :] = tmin[m]
        datasets['ds_' + GRIDNAME[i]]['WIND_HGT_CANYON'][m, :, :] = wi_cesm[m]
        
    for j in range(len(var1)):
        for m in range(10):
            if j < 4:
                datasets['ds_' + GRIDNAME[i]][var1[j]][:, m, :, :] = ucp1_cesm[j][m]
                datasets['ds_' + GRIDNAME[i]][var1_1[j]][:, m, :, :] = ucp1_cesm[j][m]
            else:
                datasets['ds_' + GRIDNAME[i]][var1[j]][m, :, :] = ucp1_cesm[j][m]
    
    for j in range(len(var2)):
        for m in range(10):
            datasets['ds_' + GRIDNAME[i]][var2[j]][m, :, :] = ucp2_cesm[j][m]
            
            
    for label, change_factors in enumerate(thermal_combinations):
        modified_datasets = {}
        for j, factor in enumerate(change_factors):
            for m in range(10):
                modified_data = ucp3_cesm[j][m] * factor
                datasets['ds_' + GRIDNAME[i]][var3[j]][:, m, :, :] = modified_data
    
        output = output_path + GRIDNAME[i] + '/surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c240510_ther_' + str(label) + '.nc'
        output_dir = os.path.dirname(output)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        if os.path.exists(output):
            os.remove(output)
        datasets['ds_' + GRIDNAME[i]].to_netcdf(output)         

In [8]:
# indoor temperature parameters
# CESM LCZ-table export surface data
temperature_combinations = list(itertools.product([1.2,1.15,1.1,1.05,0.95,0.9,0.85,0.8], repeat=1))
for i in range(len(GRIDNAME)):
    for m in range(10):
        # use the default 
        datasets['ds_' + GRIDNAME[i]]['WALL_TO_PLAN_AREA_RATIO'][m, :, :] = default['ds_' + GRIDNAME[i]]['WALL_TO_PLAN_AREA_RATIO'][2, :, :]
        datasets['ds_' + GRIDNAME[i]]['NLEV_IMPROAD'][m, :, :] = nlev[m]
        datasets['ds_' + GRIDNAME[i]]['PCT_URBAN'][m, :, :] = lcz_type[m]
        datasets['ds_' + GRIDNAME[i]]['WIND_HGT_CANYON'][m, :, :] = wi_cesm[m]
        
    for j in range(len(var1)):
        for m in range(10):
            if j < 4:
                datasets['ds_' + GRIDNAME[i]][var1[j]][:, m, :, :] = ucp1_cesm[j][m]
                datasets['ds_' + GRIDNAME[i]][var1_1[j]][:, m, :, :] = ucp1_cesm[j][m]
            else:
                datasets['ds_' + GRIDNAME[i]][var1[j]][m, :, :] = ucp1_cesm[j][m]
                
    for j in range(len(var2)):
        for m in range(10):
            datasets['ds_' + GRIDNAME[i]][var2[j]][m, :, :] = ucp2_cesm[j][m]
            
            
    for j in range(len(var3)):
        for m in range(10):
            datasets['ds_' + GRIDNAME[i]][var3[j]][:, m, :, :] = ucp3_cesm[j][m]
            
    for label, change_factors in enumerate(temperature_combinations):
        modified_datasets = {}
        for j, factor in enumerate(change_factors):
            for m in range(10):
                modified_data = tmin_c[m] * factor
                datasets['ds_' + GRIDNAME[i]]['T_BUILDING_MIN'][m, :, :] = modified_data + 273.15
    
        output = output_path + GRIDNAME[i] + '/surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c240510_temp_' + str(label) + '.nc'
        if os.path.exists(output):
            os.remove(output)
        datasets['ds_' + GRIDNAME[i]].to_netcdf(output)         

In [96]:
modified_data

11.08000000000002

# Description

In [97]:
# morphological change index
morphological_combinations = list(itertools.product([1.2, 0.8], repeat=len(var2)))
data_list_morp = []       
            
for label, change_factors in enumerate(morphological_combinations):
    label_longname = '_'.join(['{}_{}'.format(var2[j],factor) for j, factor in enumerate(change_factors)])
    data_list_morp.append({'Index': label, 'label_longname': label_longname, 'Type': 'morp'})
df_morp = pd.DataFrame(data_list_morp) 

radiative_combinations = list(itertools.product([1.2, 0.8], repeat=8))
data_list_radi = [] 
for label, change_factors in enumerate(radiative_combinations): # for albedo
    label_longname = '_'.join(['{}_{}'.format(var1[j],factor) for j, factor in enumerate(change_factors)])
    data_list_radi.append({'Index': label, 'label_longname': label_longname, 'Type': 'radi'})
df_radi = pd.DataFrame(data_list_radi)     

thermal_combinations = list(itertools.product([1.2, 0.8], repeat=len(var3)))
data_list_ther = [] 
for label, change_factors in enumerate(thermal_combinations):
    label_longname = '_'.join(['{}_{}'.format(var3[j],factor) for j, factor in enumerate(change_factors)])
    data_list_ther.append({'Index': label, 'label_longname': label_longname, 'Type': 'ther'})
df_ther = pd.DataFrame(data_list_ther) 

temperature_combinations = list(itertools.product([1.2,1.15,1.1,1.05, 0.95,0.9,0.85,0.8], repeat=1))
data_list_temp = [] 
for label, change_factors in enumerate(temperature_combinations):
    label_longname = '_'.join(['{}_{}'.format('T_BUILDING_MIN',factor) for j, factor in enumerate(change_factors)])
    data_list_temp.append({'Index': label, 'label_longname': label_longname, 'Type': 'temp'})
df_temp = pd.DataFrame(data_list_temp) 

combined_df = pd.concat([df_morp, df_radi, df_ther, df_temp], ignore_index=True)

In [98]:
combined_df.to_csv('index_list.csv')

In [99]:
combined_df

,Index,label_longname,Type
0,0,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
1,1,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
2,2,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
3,3,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
4,4,CANYON_HWR_1.2_HT_ROOF_1.2_THICK_ROOF_1.2_THIC...,morp
...,...,...,...
387,3,T_BUILDING_MIN_1.05,temp
388,4,T_BUILDING_MIN_0.95,temp
389,5,T_BUILDING_MIN_0.9,temp
390,6,T_BUILDING_MIN_0.85,temp
